# LCAv - Dev Notebook

In [1]:
# Libraries import
import sys
import os
import os.path as pth
sys.path.insert(0, '../..')
    
from lcav.lca_problem import *
from lcav.io.configuration import *
from lcav.gui.plots import *

# Paths
#os.chdir('./notebook')
DATA_FOLDER = './data'
CONFIGURATION_FILE = pth.join(DATA_FOLDER, 'configuration.yaml')

## 1. Create an LCA problem from a configuration file

### 1.a) Load configuration file and create LCA model

In [2]:
problem = LCAProblemConfigurator(CONFIGURATION_FILE).get_problem()

In [3]:
problem.list_databases()

In [4]:
problem.list_parameters()

In [5]:
problem.list_processes()

In [7]:
# Plot process tree: REPLACE WITH FUNCTIONS FROM ACTIVITY BROWSER OF BW2 ?
process_tree(problem)

### 1.b) Compile LCIA functions and/or run LCIA

In [8]:
problem.compute_lcia(
    a = 2.0,
    b = 1.5,
    # ... default values will be set for parameters not provided by the user
)

# passing a dictionnary of parameters with ** also works: 
# params_dict = {'a': 2.0, 'b': 1.5}
# compute_lcia(**params_dict)

In [37]:
problem.list_lcia_functions()

In [50]:
import IPython.display as disp

for expr in problem.lambdas:
    disp.display(expr)

## 2. Reduced problem: for very fast computation but limited capabilities

This reduced problem does not require EcoInvent and to setup a brightway/lca_algebraic project.<br>
It is generated by a full LCA problem (see LCAProblem class).<br>
The LCIA relies on previously compiled expressions of the impacts.

In [ ]:
# Generate from full LCA problem
reduced_problem = problem.generate_reduced_problem()

In [ ]:
# Or load existing reduced problem
FILE_PATH = pth.join(DATA_FOLDER, 'reduced_problem.pickle')
reduced_problem = LCAProblemReduced(load_from_file=FILE_PATH)

In [ ]:
# LCIA expressions
reduced_problem.list_lcia_functions()

In [ ]:
# Process tree (this is more of a picture of the original LCA problem)
process_tree(reduced_problem)

In [ ]:
# Run LCIA
res = reduced_problem.compute_lcia(
    a = [1.0, 3.0],
    switch_process_switch_param = 'backgroundprocessa',
    unused_params_will_be_ignored = 1.0,
    # ...
)
res

In [ ]:
# Save reduced problem in .pickle file for future use
FILE_PATH = pth.join(DATA_FOLDER, 'reduced_problem.pickle')
reduced_problem.save(FILE_PATH)

## 3. In-loop LCAv calculations: example

### 3.a) Case study

In [ ]:
from scipy.optimize import minimize

# Define the Rosenbrock function
def rosenbrock(x):
    a = 1
    b = 100
    return (a - x[0])**2 + b * (x[1] - x[0]**2)**2

# Initial guess
initial_guess = [0, 0]

# Perform optimization
result = minimize(rosenbrock, initial_guess, method='BFGS')

# Print the result
print("Optimal parameters:", result.x)
print("Minimum value:", result.fun)

### 3.b) Optimization with LCIA (full problem)

The first call to `compute_lcia()` compiles the LCIA expressions so takes a while, but the following calls are much faster.

In [ ]:
# Setup LCA problem defined in configuration file
print('Setting up the LCA problem.')
problem = LCAProblemConfigurator(CONFIGURATION_FILE).get_problem()
problem._compile_lcia_functions()  # not necessary. If not done here, this will be automatically achieved at the first call of compute_lcia()
print('LCA problem configured and compiled. Now running optimization.')

# Define the Rosenbrock function with LCA calculations
def rosenbrock(x):
    a = 1
    b = 100
    func = (a - x[0])**2 + b * (x[1] - x[0]**2)**2
    
    func_lca = problem.compute_lcia(
        a = a,
        b = b,
        c = x[0],
        d = x[1],
        f = 2.0,
        g = x[0] + x[1],
        scaler_value = 10.0,
        switch_process_switch_param = 'backgroundprocessa'
    ).values[0][0]  # results for first impact method
    
    return func_lca

# Initial guess
initial_guess = [0, 0]

# Perform optimization
result = minimize(rosenbrock, initial_guess, method='BFGS')

# Print the result
print("Optimal parameters:", result.x)
print("Minimum value:", result.fun)

### 3.c) Optimization with reduced LCA problem

The compilation of LCIA functions is already done so that no initialization time is required.

In [ ]:
# Load existing reduced problem
FILE_PATH = pth.join(DATA_FOLDER, 'reduced_problem.pickle')
reduced_problem = LCAProblemReduced(load_from_file=FILE_PATH)

# Define the Rosenbrock function with LCA calculations
def rosenbrock(x):
    a = 1
    b = 100
    func = (a - x[0])**2 + b * (x[1] - x[0]**2)**2
    
    func_lca = reduced_problem.compute_lcia(
        a = a,
        b = b,
        c = x[0],
        d = x[1],
        f = 2.0,
        g = x[0] + x[1],
        scaler_value = 10.0,
        switch_process_switch_param = 'backgroundprocessa'
    ).values[0][0]  # results for first impact method
    
    return func_lca

# Initial guess
initial_guess = [0, 0]

# Perform optimization
result = minimize(rosenbrock, initial_guess, method='BFGS')

# Print the result
print("Optimal parameters:", result.x)
print("Minimum value:", result.fun)